In [42]:
import numpy as np
import pandas as pd
import plotly.express as px
import sympy as sp


class Person:
    def __init__(self, name: str, foresight: int):
        self.name = name
        self.foresight = foresight
        self.decision_list = []

    def append_optimal_choice(
        self, choice_array: np.ndarray, decision_number: int, number_of_decisions: int
    ):
        # Get the row index of the highest sum of values in the choice array up to the person's "foresight" index
        highest_value_index = np.argmax(
            np.sum(choice_array[:, : self.foresight], axis=1)
        )

        # Extract this row from the choice array
        optimal_choice = choice_array[highest_value_index, :]

        # Pad initial and final values with nan
        optimal_choice_padded = np.pad(
            optimal_choice,
            (decision_number, number_of_decisions - decision_number),
            mode="constant",
            constant_values=np.nan,
        )

        # Append to decision list
        self.decision_list.append(optimal_choice_padded)

    def convert_decision_list_to_df(self):
        # Swap rows and columns
        decision_array = np.array(self.decision_list)
        decision_array_inverted = decision_array.T
        decision_df = pd.DataFrame(decision_array_inverted)
        self.decision_df = decision_df
        return decision_df

    def calc_life_score(self):
        # Sum columns
        self.decision_df["decision_sum"] = self.decision_df.sum(axis=1)

        # Calculate life score as the cumulative sum of the decision sums
        self.decision_df["life_score"] = self.decision_df["decision_sum"].cumsum()
        return self.decision_df


class LifeSimulation:
    def __init__(
        self,
        people: list[Person],
        n_choices: int,
    ):
        self.people = people
        self.n_choices = n_choices

    def add_person(self, person: Person):
        self.people.append(person)

    def get_people(self) -> list[Person]:
        return self.people

    def get_person_by_name(self, name: str) -> Person | None:
        for person in self.people:
            if person.name == name:
                return person
        return None

    def get_number_of_people(self) -> int:
        return len(self.people)

    def generate_random_choice_function(self, seed=None):
        x = sp.symbols("x")

        if seed:
            np.random.seed(seed)
        m1 = np.random.uniform(0, 2)
        m2 = np.random.uniform(5, 8)
        s1 = np.random.uniform(0.1, 1)
        s2 = np.random.uniform(0.1, 1)
        a1 = np.random.uniform(-1, 1)
        a2 = np.random.uniform(-1, 1)

        y = a1 * sp.exp(-((x - m1) ** 2) / (2 * s1**2)) + a2 * sp.exp(
            -((x - m2) ** 2) / (2 * s2**2)
        )

        return {"m1": m1, "m2": m2, "s1": s1, "s2": s2, "a1": a1, "a2": a2, "expr": y}

    def generate_choice_array(self, n_choices: int):
        x = sp.symbols("x")

        # Generate n random functions (seeds for reproducibility)
        functions = [
            self.generate_random_choice_function(seed=i) for i in range(n_choices)
        ]

        # Create x values (100 points from 0 to 100)
        x_vals = np.linspace(0, 100, 100)

        # Build NumPy array: n rows (functions) × 100 columns (y values)
        choice_array = np.zeros((n_choices, 100))
        for i, func in enumerate(functions):
            y_func = sp.lambdify(x, func["expr"], "numpy")
            choice_array[i, :] = y_func(x_vals)
        return choice_array

    def simlulate_life(self, number_of_decisions: int):
        for person in self.people:
            for decision_number in range(number_of_decisions):
                choice_array = self.generate_choice_array(self.n_choices)
                person.append_optimal_choice(
                    choice_array, decision_number, number_of_decisions
                )
            person.convert_decision_list_to_df()
            person.calc_life_score()


def main():
    # Define people
    people = [
        Person("Amy", 2),
        Person("Barry", 8),
    ]

    # Define simulation configuration
    n_choices = 100
    number_of_decisions = 500

    # Run simulation
    simulation = LifeSimulation(people, n_choices)

    # Check example choice array
    """ choice_array = simulation.generate_choice_array(n_choices=3)
    print(choice_array.shape)

    fig = px.line(choice_array.T)
    fig.show() """

    # Run simulation
    simulation.simlulate_life(number_of_decisions)

    # Combine decision_df["life_score"] into single dataframe
    life_score_df = pd.DataFrame(
        {
            k: v
            for k, v in zip(
                [person.name for person in simulation.people],
                [person.decision_df["life_score"] for person in simulation.people],
            )
        }
    )

    print(life_score_df)

    # Plot life_score lines
    fig = px.line(life_score_df)
    fig.show()


if __name__ == "__main__":
    main()

            Amy        Barry
0      0.924173     0.007817
1      2.347638     0.251522
2      3.829998     0.912103
3      5.313873     1.619396
4      6.797619     2.387768
..          ...          ...
595  628.015146  1105.314666
596  628.015146  1105.314666
597  628.015146  1105.314666
598  628.015146  1105.314666
599  628.015146  1105.314666

[600 rows x 2 columns]
